In [1]:
import os, io
import pandas as pd
import re
import numpy as np

### Clean Crypto data and save in Data/Cleaned/Crypto/

In [2]:

os.makedirs("data/cleaned/crypto", exist_ok=True)

def clean_file(raw_path, out_path, name):
    """
    Reads raw CSV at raw_path (which has extra header lines),
    finds the first actual date row, and writes a clean CSV
    with exactly two columns: Date, Close.
    """
    # 1) Locate first "YYYY-MM-DD" line
    first = 0
    with open(raw_path, 'r') as f:
        for i, line in enumerate(f):
            if re.match(r'\d{4}-\d{2}-\d{2}', line.split(',')[0]):
                first = i
                break

    # 2) Read from that line onward
    df = pd.read_csv(
        raw_path,
        skiprows=first,
        names=["Date", "Close"],
        usecols=[0, 1],
        parse_dates=["Date"],
        index_col="Date"
    )

    # 3) Save cleaned CSV
    df.to_csv(out_path)
    print(f"Cleaned {name}: {len(df)} rows → {out_path}")

if __name__ == "__main__":
    clean_file("data/crypto/btc.csv", "data/cleaned/crypto/btc.csv", "BTC")
    clean_file("data/crypto/eth.csv", "data/cleaned/crypto/eth.csv", "ETH")


Cleaned BTC: 2556 rows → data/cleaned/crypto/btc.csv
Cleaned ETH: 2556 rows → data/cleaned/crypto/eth.csv


### Clean and Save Data/cleaned/industry/

In [3]:


# 1) Ensure output folder exists
os.makedirs("data/cleaned", exist_ok=True)

# 2) Read raw lines
with open("data/indices/10_Industry_Portfolios_Daily.csv", "r") as f:
    raw = f.read().splitlines()

# 3) Find end of value-weighted block
stop = next(i for i, row in enumerate(raw)
            if "Average Equal Weighted Returns" in row)
val = raw[:stop]

# 4) Locate the header line (starts with ",NoDur")
hdr_idx = next(i for i, row in enumerate(val) if row.startswith(",NoDur"))
header = val[hdr_idx].lstrip(",").split(",")

# 5) Data lines follow immediately after the header
data_lines = val[hdr_idx+1:]

# 6) Parse into DataFrame
df = pd.read_csv(
    io.StringIO("\n".join(data_lines)),
    names=header,
    parse_dates=[0],
    index_col=0,
)

# 7) Coerce to numeric (turn “-99.99” into NaN)
df = df.apply(pd.to_numeric, errors="coerce")

# 8) Extract & rename our four sectors, convert %→decimal
ind = df[["NoDur","Manuf","HiTec","Hlth"]]
ind = ind.rename(columns={"NoDur":"Consumer"}) / 100.0

# 9) Build cumulative price indices (start at 1.0)
price_ind = (1 + ind).cumprod()

# 9.5) Keep only data between 2018-01-01 and 2024-12-30
price_ind = price_ind.loc["2018-01-01":"2024-12-30"]

# 10) Add a Date column from the index for clarity
price_ind_with_date = price_ind.copy()
price_ind_with_date["Date"] = price_ind_with_date.index

# 11) Move Date to the first column
cols = ["Date"] + [col for col in price_ind_with_date.columns if col != "Date"]
price_ind_with_date = price_ind_with_date[cols]

# 12) Save cleaned CSV
out_path = "data/cleaned/industry/industries.csv"
price_ind_with_date.to_csv(out_path, index=False)
print(f"Saved cleaned industry indices to {out_path}")


Saved cleaned industry indices to data/cleaned/industry/industries.csv


/var/folders/jp/82zjtkm90vgbycmw30_2kztm0000gn/T/ipykernel_34830/773625436.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


### Checking any missing cell

In [4]:
# 1) Load the industries file, telling pandas that the first column is the Date index
ind = pd.read_csv(
    "data/cleaned/industry/industries.csv",
    index_col=0,          # treat first column as index
    parse_dates=True      # parse that index as dates
)

# 2) Rename the columns to lowercase
ind.columns = ["Consumer", "Manuf", "HiTec", "Hlth"]

# 3) Check for missing data
print(ind.isna().sum())


Consumer    0
Manuf       0
HiTec       0
Hlth        0
dtype: int64


In [5]:
# 1) Ensure output folder
os.makedirs("data/cleaned", exist_ok=True)

# 2) Load cleaned CSVs
btc        = pd.read_csv("data/cleaned/crypto/btc.csv", parse_dates=["Date"], index_col="Date")["Close"]
eth        = pd.read_csv("data/cleaned/crypto/eth.csv", parse_dates=["Date"], index_col="Date")["Close"]
industries = pd.read_csv("data/cleaned/industry/industries.csv", parse_dates=["Date"], index_col="Date")

# 3) Rename series
btc.name = "btc"
eth.name = "eth"

# 4) Join into one DataFrame
prices = industries.join([btc, eth], how="left").ffill()

# 5) Save merged prices
prices.to_csv("data/cleaned/all_prices.csv")
print("Merged prices saved to data/cleaned/all_prices.csv")

# 6) Compute log-returns using numpy
log_returns = np.log(prices / prices.shift(1))

# 7) Save log-returns (log_returns is a DataFrame, so .to_csv works)
log_returns.to_csv("data/cleaned/log_returns.csv") # type: ignore
print("Log-returns saved to data/cleaned/log_returns.csv")


Merged prices saved to data/cleaned/all_prices.csv
Log-returns saved to data/cleaned/log_returns.csv
